# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load csv, delete useless extra column

csv_file = os.path.join("../output_data/cities.csv")
cities_df = pd.read_csv(csv_file)
del cities_df["Unnamed: 0"]
cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed (mph)
0,Atuona,-9.8000,-139.0333,79.97,75,37,12.82
1,Busselton,-33.6500,115.3333,57.00,93,90,1.01
2,Jamestown,42.0970,-79.2353,71.60,49,75,16.11
3,Saint Paul Harbor,57.7900,-152.4072,39.20,87,90,13.80
4,Hong Kong,22.2855,114.1577,75.99,78,100,10.04
...,...,...,...,...,...,...,...
556,Alofi,-19.0595,-169.9187,75.20,83,98,11.50
557,Tiarei,-17.5333,-149.3333,78.80,73,20,2.30
558,Kanniyākumāri,8.0833,77.5667,83.16,76,50,4.07
559,Bulawayo,-20.1500,28.5833,56.59,67,52,11.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gkey
gmaps.configure(api_key=g_key)

In [4]:
# define variable from df
humidity = cities_df["Humidity"].astype(float)

# use latitude and longitude as locations
locations_df = cities_df[["Latitude", "Longitude"]]

# use max humidity as the weight
#max_humidity = humidity.max()

In [5]:
# create gmap figure
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity, dissipating=False, point_radius=5, max_intensity=100)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vacation_cities_df = cities_df.loc[(cities_df["Max Temp"] > 75) 
                                   & (cities_df["Max Temp"] < 85) 
                                   & (cities_df["Humidity"] < 65) 
                                   & (cities_df["Cloudiness"] < 50) 
                                   & (cities_df["Wind Speed (mph)"] < 10), :]
vacation_cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed (mph)
38,Saint George,37.1041,-113.5841,79.00,16,1,5.75
64,Karachi,24.9056,67.0822,84.20,51,0,9.22
147,Astorga,-23.2325,-51.6656,84.20,42,20,9.22
198,Nuevo Ideal,24.8875,-105.0728,75.27,14,14,2.08
228,Saldanha,-33.0117,17.9442,75.20,22,0,6.91
237,Caravelas,-17.7125,-39.2481,79.16,56,31,8.75
343,Ixtapa,20.7000,-105.2000,80.60,39,1,9.22
361,Sylvania,41.7189,-83.7130,80.60,31,1,8.05
417,Porto Seguro,-16.4497,-39.0647,80.60,57,40,8.05
447,Boyuibe,-20.4167,-63.2833,82.40,47,40,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# create new hotel df 
hotel_df = vacation_cities_df

In [8]:
# loop through df and use google API
for index, row in hotel_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        # make parameter dictionary
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
        
        latitude = row["Latitude"]
        longitude = row["Longitude"]

        params["location"] = f"{latitude}, {longitude}"
        
        # request from google API
        hotel_data = requests.get(url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    
    # get rid of any NaN data before showing new dataframe   
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

/Users/carleyengleson/opt/anaconda3/envs/My_Python_Env/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/carleyengleson/opt/anaconda3/envs/My_Python_Env/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed (mph),Hotel Name
38,Saint George,37.1041,-113.5841,79.00,16,1,5.75,Comfort Inn Saint George North
64,Karachi,24.9056,67.0822,84.20,51,0,9.22,Hotel Galaxy
147,Astorga,-23.2325,-51.6656,84.20,42,20,9.22,Astorga Park Hotel
198,Nuevo Ideal,24.8875,-105.0728,75.27,14,14,2.08,Hotel Las Águilas
228,Saldanha,-33.0117,17.9442,75.20,22,0,6.91,Saldanha Bay Hotel
237,Caravelas,-17.7125,-39.2481,79.16,56,31,8.75,Mar Doce de Abrolhos
343,Ixtapa,20.7000,-105.2000,80.60,39,1,9.22,Club Regina Puerto Vallarta
361,Sylvania,41.7189,-83.7130,80.60,31,1,8.05,Wingate by Wyndham Sylvania/Toledo
417,Porto Seguro,-16.4497,-39.0647,80.60,57,40,8.05,Maitei Hotel
447,Boyuibe,-20.4167,-63.2833,82.40,47,40,4.61,Residencial Eluzai


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))